In [145]:
import imageParse as imgP
import imageSent as imgS
import numpy as np
import pandas as pd
import tensorflow as tf

print(tf.__version__)

2.7.0


In [166]:
tf.config.run_functions_eagerly(False)

In [159]:
tf.data.experimental.enable_debug_mode()

In [160]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [161]:
abs_X0, abs_Y0, abs_X1, abs_Y1 = imgP.parseData_Abs()
print(abs_X0.shape)
print(abs_X1.shape)

(224, 255, 255, 3)
(56, 255, 255, 3)


In [162]:
abs_X0 = imgP.input_prep_fn(abs_X0)
abs_X1 = imgP.input_prep_fn(abs_X1)
print(abs_X0.shape)
print(abs_X1.shape)

(224, 32, 32, 3)
(56, 32, 32, 3)


In [167]:
art_X0, art_Y0, art_X1, art_Y1 = imgP.parseData_Art()
print(art_X0.shape)
print(art_X1.shape)

(644, 255, 255, 3)
(161, 255, 255, 3)


In [168]:
art_X0 = imgP.input_prep_fn(art_X0)
art_X1 = imgP.input_prep_fn(art_X1)
print(art_X0.shape)
print(art_X1.shape)

(644, 32, 32, 3)
(161, 32, 32, 3)


In [169]:
VAD_map = np.array([
    [1,0.735,0.772], #0: 'happy'
    [0.918,0.61,0.566], #1: 'funny'
    [0.225,0.333,0.149], #2: 'sad'
    [0.63,0.52,0.509], #3: 'tender'
    [0.95,0.792,0.789], #4: 'exciting'
    [0.122,0.83,0.604], #5: 'angry'
    [0.062,0.952,0.528], #6: 'scary'
])

VAD_pd = pd.DataFrame(
    columns = ["valence", "arousal", "dominance"], 
    data = VAD_map, 
    index = ['happy', 'funny', 'sad', 'tender', 
             'exciting', 'angry', 'scary'])

In [175]:
abs_model = imgS.ImageSentModel(name='abs')
abs_model((abs_X0[:5], abs_Y0[:5]))
abs_model.summary()

Model: "abs"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_48 (Conv2D)          multiple                  224       
                                                                 
 conv2d_49 (Conv2D)          multiple                  1168      
                                                                 
 conv2d_50 (Conv2D)          multiple                  4640      
                                                                 
 max_pooling2d_18 (MaxPoolin  multiple                 0         
 g2D)                                                            
                                                                 
 flatten_18 (Flatten)        multiple                  0         
                                                                 
 dense_36 (Dense)            multiple                  491580    
                                                               

In [176]:
optimizer = tf.keras.optimizers.Adam(5e-3)
abs_model.compile(optimizer=optimizer, VAD_map=VAD_map)

In [177]:
abs_Y0_VAD = VAD_pd.loc[abs_Y0]
abs_X0 = abs_X0 + tf.random.normal(abs_X0.shape, mean = 0, stddev = 0.05, dtype = tf.float32)

In [178]:
abs_model.fit(
    (abs_X0, abs_Y0_VAD), abs_Y0_VAD,
    epochs     = 5,
    batch_size = 8,
);

Epoch 1/5
28/28 [==============================] - 1s 12ms/step - mse_loss: 0.1960 - acc: 0.1339
Epoch 2/5
28/28 [==============================] - 0s 14ms/step - mse_loss: 0.1994 - acc: 0.1339
Epoch 3/5
28/28 [==============================] - 0s 11ms/step - mse_loss: 0.1994 - acc: 0.1339
Epoch 4/5
28/28 [==============================] - 0s 11ms/step - mse_loss: 0.1994 - acc: 0.1339
Epoch 5/5
28/28 [==============================] - 0s 11ms/step - mse_loss: 0.1994 - acc: 0.1339


In [183]:
abs_Y1_VAD = VAD_pd.loc[abs_Y1]
print(abs_Y1_VAD.shape)

(56, 3)


In [188]:
abs_model.evaluate(
    (abs_X1, abs_Y1_VAD), abs_Y1_VAD,
    batch_size = 8,
);

7/7 [==============================] - 0s 4ms/step - mse_loss: 0.2292 - acc: 0.0714


In [189]:
art_model = imgS.ImageSentModel(name='art')
art_model((art_X0[:5], art_Y0[:5]))
art_model.summary()

Model: "art"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_51 (Conv2D)          multiple                  224       
                                                                 
 conv2d_52 (Conv2D)          multiple                  1168      
                                                                 
 conv2d_53 (Conv2D)          multiple                  4640      
                                                                 
 max_pooling2d_19 (MaxPoolin  multiple                 0         
 g2D)                                                            
                                                                 
 flatten_19 (Flatten)        multiple                  0         
                                                                 
 dense_38 (Dense)            multiple                  491580    
                                                               

In [190]:
art_model.compile(optimizer=optimizer, VAD_map=VAD_map)

In [191]:
art_Y0_VAD = VAD_pd.loc[art_Y0]
art_X0 = art_X0 + tf.random.normal(art_X0.shape, mean = 0, stddev = 0.05, dtype = tf.float32)

In [192]:
art_model.fit(
    (art_X0, art_Y0_VAD), art_Y0_VAD,
    epochs     = 5,
    batch_size = 8,
);

Epoch 1/5
2
2
81/81 [==============================] - 2s 12ms/step - mse_loss: 0.2933 - acc: 0.1265
Epoch 2/5
81/81 [==============================] - 1s 11ms/step - mse_loss: 0.2953 - acc: 0.1281
Epoch 3/5
81/81 [==============================] - 1s 12ms/step - mse_loss: 0.2955 - acc: 0.1265
Epoch 4/5
81/81 [==============================] - 1s 12ms/step - mse_loss: 0.2953 - acc: 0.1265
Epoch 5/5
81/81 [==============================] - 1s 16ms/step - mse_loss: 0.2956 - acc: 0.1296


In [193]:
art_Y1_VAD = VAD_pd.loc[art_Y1]
print(art_Y1_VAD.shape)

(161, 3)


In [194]:
art_model.evaluate(
    (art_X1, art_Y1_VAD), art_Y1_VAD,
    batch_size = 8,
);

21/21 [==============================] - 0s 4ms/step - mse_loss: 0.3028 - acc: 0.1131
